# NYC Buildings

Many plotting libraries can handle collections of polygons, e.g. [Bokeh](https://docs.bokeh.org/en/latest/docs/gallery/texas.html) or [HoloViews+Bokeh](http://holoviews.org/gallery/demos/bokeh/choropleth_data_link.html). However, because browser-based libraries like Bokeh and Plotly send all the polygon data to the browser, they can struggle when either the collections or the polygons themselves get large. Even natively in Python, typical formats like Shapely for representing polygons scale poorly to large polygon collections, because each polygon is wrapped up as a separate Python object, leading to a lot of duplicated storage overhead when many polygons of the same type are defined.

If you want to work with lots of polygons, here you can see how to use [SpatialPandas](https://github.com/holoviz/spatialpandas) and Dask to represent polygons efficiently in memory, fastparquet to represent them efficiently on disk, and [Datashader](https://datashader.org) to render them quickly in a web browser. This notebook also demonstrates how to support hovering for datashaded polygons, with Bokeh overlaying a single vector-based representation of a polygon where the mouse cursor is, while all the rest are sent to the browser only as rendered pixels. That way hover and other interactive features can be supported fully without ever needing to transfer large amounts of data or store them in the limited memory of the web browser tab.  

This example plots the outlines of all the buildings in New York City. See
[nyc.gov](https://www1.nyc.gov/site/doitt/residents/gis-2d-data.page) for the original data and its description.

In [ ]:
import holoviews as hv
import colorcet as cc
import datashader as ds
import spatialpandas as spd
import spatialpandas.io

from dask.diagnostics import ProgressBar
from holoviews.operation.datashader import (
    rasterize, datashade, inspect_polygons
)

hv.extension('bokeh')

In [ ]:
ddf = spd.io.read_parquet_dask('./data/nyc_buildings.parq').persist()

Now we compute the top categories and drop everything else:

In [ ]:
cats = list(ddf.type.value_counts().compute().iloc[:10].index.values) + ['unknown']
ddf['type'] = ddf.type.replace({None: 'unknown'})
ddf = ddf[ddf.type.isin(cats)]
ddf['type'] = ddf['type'].astype('category').cat.as_known()

with ProgressBar():
    ddf = ddf.build_sindex().persist()

Next we build a legend for the categories and declare a tile source as backdrop:

In [ ]:
colors    = cc.glasbey_bw_minc_20_maxl_70
color_key = {cat: tuple(int(e*255.) for e in colors[i]) for i, cat in enumerate(cats)}
legend    = hv.NdOverlay({k: hv.Points([0,0], label=str(k)).opts(
                                         color=cc.rgb_to_hex(*v), size=0, apply_ranges=False)
                          for k, v in color_key.items()}, 'Type')

tiles = hv.element.tiles.CartoLight().opts(
    min_height=500, responsive=True, xaxis=None, yaxis=None)

Now we put it all together, declaring a `Polygons` element from our data, datashade them and use the `inspect_polygons` operation to allow us to hover on the data:

In [ ]:
polys = hv.Polygons(ddf, vdims='type')

shaded = datashade(polys, color_key=color_key, aggregator=ds.by('type', ds.any()))

hover = inspect_polygons(shaded).opts(fill_color='red', tools=['hover'])

tiles * shaded * legend * hover

Finally we will plot each category of buildings separately:

In [ ]:
hv.NdLayout({
    cat: hv.element.tiles.CartoLight() * rasterize(polys.select(type=cat), aggregator='any') for cat in cats
}, 'Type').opts('Image', width=250, height=400, xaxis=None, yaxis=None).cols(4)